# Keysight P9375A Testing

## Instructions

For this instrument to work, both remote access and remote drive access need to be enabled. 

To enable these, navigate to it via the menu bar for the soft panel: `Utility -> System -> System Setup -> Remote Interface`.

Make sure both are checked.

## Environment

In [ ]:
import pyvisa

from keysight_p9375a import KeysightP9375A

# unnecessary but kept for now
# os.add_dll_directory(r"C:\Program Files (x86)\Keysight\IO Libraries Suite\bin")
# os.add_dll_directory(r"C:\Program Files\Keysight\IO Libraries Suite\bin")

rm = pyvisa.ResourceManager()

In [ ]:
# create vna object using hislip
vna = KeysightP9375A(rm.open_resource("TCPIP0::127.0.0.1::hislip0::INSTR"))

In [ ]:
vna.id

In [ ]:
vna.ch_1.scan_points

In [ ]:
vna.ch_1.scan_points = 1601

In [ ]:
vna.ch_1.averaging_count

In [ ]:
vna.ch_1.averaging_enabled

In [ ]:
vna.ch_1.averaging_mode

In [ ]:
vna.ch_1.IFBW

In [ ]:
vna.ch_1.correction_enabled

In [ ]:
vna.ch_1.dwell_time

In [ ]:
vna.ch_1.power

In [ ]:
vna.ch_1.start_frequency

In [ ]:
vna.ch_1.stop_frequency

In [ ]:
vna.reset()

In [ ]:
s_parms = ["S11", "S21", "S22", "S12"]

for parm in s_parms:
    vna.write(f"CALC:PAR:DEF {parm}, {parm}")

In [ ]:
vna.ask("CALC1:MEAS1:DATA:SNP?").split(",")

In [ ]:
vna.ask("CALCulate1:PARameter:CATalog:EXTended?")

In [ ]:
vna.ch_1.save_snp_touchstone(
    single_sweep=True,
    remote_path=r"C:\Users\Public\Documents\DUT_011.s1p",
    ports="2",
    channel=1,
    meas=1,
    snp_format="RI",
    fetch_to=r".\data\DUT_011.s1p",
)

In [ ]:
# from __future__ import annotations

from pathlib import Path

from pymeasure.instruments import Instrument


def save_snp_touchstone(
    vna: Instrument,
    *,
    remote_path: str,
    ports: str = "1,2",
    channel: int = 1,
    meas: int = 1,
    snp_format: str = "RI",  # "RI", "MA", "DB", or "AUTO"
    single_sweep: bool = True,
    fetch_to: str | Path | None = None,
    timeout_ms: int = 120_000,
) -> Path | None:
    """
    Save S-parameter data from the selected measurement to a Touchstone SnP file
    on the VNA PC filesystem, and optionally transfer it back to the controller.

    Parameters
    ----------
    vna:
        A PyMeasure Instrument already connected via VISA.
    remote_path:
        Full path *on the VNA controller PC*, e.g.
        r'C:\\Users\\Public\\Documents\\MyData.s2p'
    ports:
        Comma/space delimited list of ports in quotes per SCPI, e.g. "1,2" or "1,2,3,4".
    channel, meas:
        Channel and measurement index used by the SCPI command.
    snp_format:
        Sets MMEM:STORe:TRACe:FORMat:SNP (RI/MA/DB/AUTO).
    single_sweep:
        If True, disables continuous triggering and runs one sweep before saving.
        (Keysight recommends triggering a single measurement then letting the channel go to Hold
        before saving.)
    fetch_to:
        If provided, reads the saved file back using MMEM:TRANsfer? and writes it locally.
        Note: requires “Enable Remote Drive Access” in the VNA Remote Interface dialog.
    timeout_ms:
        VISA timeout for long sweeps / large point counts.

    Returns
    -------
    Path to the fetched local file if fetch_to is provided; otherwise None.
    """
    # Make long operations less likely to timeout
    try:
        vna.adapter.connection.timeout = timeout_ms
    except Exception:
        pass

    # Optional: choose SnP data formatting (RI/MA/DB/AUTO)
    vna.write(f"MMEM:STOR:TRAC:FORM:SNP {snp_format}")

    if single_sweep:
        # Run one sweep and wait for completion
        vna.write("INIT:CONT OFF")
        vna.write("INIT:IMM")
        vna.ask("*OPC?")  # blocks until the sweep is complete

    # Save Touchstone file on the VNA controller PC filesystem
    # SCPI: CALCulate<cnum>:MEASure<mnum>:DATA:SNP:PORTs:SAVE "<ports>","<filename>"
    cmd = f'CALC{channel}:MEAS{meas}:DATA:SNP:PORTS:SAVE "{ports}","{remote_path}"'
    vna.write(cmd)
    vna.ask("*OPC?")  # Keysight recommends *OPC? for large point counts

    if fetch_to is None:
        return None

    # Transfer the file back to the controller as an IEEE definite-length binary block:
    # Query syntax: MMEMory:TRANsfer? <fileName>
    vna.write(f'MMEM:TRAN? "{remote_path}"')
    raw = vna.adapter.connection.read_raw()

    # Parse IEEE 488.2 definite-length block: b"#"<ndigits><nbytes><data...>
    if not raw.startswith(b"#"):
        raise RuntimeError(f"Unexpected transfer header: {raw[:64]!r}")

    ndigits = int(raw[1:2].decode("ascii"))
    nbytes = int(raw[2 : 2 + ndigits].decode("ascii"))
    data_start = 2 + ndigits
    data = raw[data_start : data_start + nbytes]

    fetch_to = Path(fetch_to)
    fetch_to.parent.mkdir(parents=True, exist_ok=True)
    fetch_to.write_bytes(data)
    return fetch_to

In [ ]:
from pymeasure.adapters import VISAAdapter
from pymeasure.instruments import Instrument

adapter = VISAAdapter("TCPIP0::127.0.0.1::hislip0::INSTR")

vna = Instrument(adapter, name="P9375A")


# Save to the VNA’s local disk and also fetch a copy to your PC
local_file = save_snp_touchstone(
    vna,
    remote_path=r"C:\Users\Public\Documents\DUT_001.s2p",
    ports="1,2",
    channel=1,
    meas=1,
    snp_format="RI",
    fetch_to=r".\data\DUT_001.s2p",
)
print("Fetched:", local_file)

In [ ]:
print(vna.ask("*IDN?"))

In [ ]:
# set data to real/imag
vna.adapter.write("MMEMory:STORe:TRACe:FORMat:SNP RI")

data = vna.adapter.query("CALC:MEAS1:DATA:SNP? 1").split(",")  # freq array, real array, imag array

freq = [float(i) for i in data[0 : int(len(data) / 3)]]
real = [float(i) for i in data[int(len(data) / 3) : int(2 * len(data) / 3)]]
imag = [float(i) for i in data[int(2 * len(data) / 3) : int(3 * len(data) / 3)]]

# result
meas = [complex(x, y) for x, y in zip(real, imag, strict=False)]

In [ ]:
meas